In [1]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")


# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust issue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("data.nii")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#model = ShoreModel(gtab)
#fit = model.fit(data, wm_mask, gm_mask, csf_mask, dti_mask, 
#                    dti_fa, dti_vecs, verbose=True)

In [3]:
fit.save("/home/olivier/Devel/test/samuel/in4michi/"+"fit.pkl")

NameError: name 'fit' is not defined

In [4]:
fit = ShoreFit.load("/home/olivier/Devel/test/samuel/in4michi/"+"fit.pkl")

In [ ]:
#fit.fiber_orientation_distribution_functions(data.get_data(), outdir = '/home/olivier/Devel/test/samuel/in4michi/bonndit_result/',verbose=True)

/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/dipy/core/geometry.py:129: RuntimeWarning: invalid value encountered in true_divide
  theta = np.arccos(z / r)
Optimization:   0%|          | 3/460800 [00:00<4:18:09, 29.75it/s]

Condition number of M: 9.246390603572164e+114


Optimization:   0%|          | 1796/460800 [01:30<6:23:50, 19.93it/s]

In [14]:
from bonndit.shore import gtab_reorient, gtab_rotate
from dipy.core.gradients import GradientTable
import numpy as np
GRADIENTS = np.array([[2500,0,0],
                      [0,2500,0],
                      [0,0,2500],
                      [0,0,0]])
gtab = GradientTable(GRADIENTS)
old_vec = np.array((0, 0, 1))
new_vec = np.array((1, 0, 0))
gtab_reorient(gtab, old_vec, new_vec).gradients

array([[    0.,     0., -2500.],
       [    0.,  2500.,     0.],
       [ 2500.,     0.,     0.],
       [    0.,     0.,     0.]])